
## 과제

- 피쳐들 타입에 따른 전처리  
	- 범주형피쳐 : 명목형/순서형 => 인코딩  
	    - 명목형(이름만 다를 뿐, 순서가 전혀 없는 데이터) => OneHotEncoding  
	    - 순서형(크다/작다, 앞/뒤, 1등/2등 개념이 있음) => OrdinalEncoding  
	- 수치형피쳐(값 자체가 숫자)  
	    - 범주형 변환 ? : 데이터가 가진 의미 체크 (숫자로 보이지만 의미가 있음. 학번/우편번호/..)  
	    - 수치형 => 스케일링 여부 (큰 값이 존재할 때, 그게 영향이 큰게 맞는지 판단)  
-> 각각 하나씩 함수로 구현하면됨  

- 타겟 컬럼에 대한 처리  
	- 범주형 => LabelEncoder 사용  
	- 수치형 => 그대로 사용  

##사용 데이터  
** titanic.csv , auto_mpg.csv  

-> 함수로 구현


In [34]:
# 모듈 로딩
import pandas as pd
import numpy as np

# ML 관련 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder, TargetEncoder


In [35]:
tDF = pd.read_csv('../DATA/titanic.csv')
aDF = pd.read_csv('../DATA/auto_mpg.csv')
display(tDF, aDF)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86,2790,15.6,82,1,ford mustang gl
394,44.0,4,97.0,52,2130,24.6,82,2,vw pickup
395,32.0,4,135.0,84,2295,11.6,82,1,dodge rampage
396,28.0,4,120.0,79,2625,18.6,82,1,ford ranger


In [36]:
tDF.info()

# 컬럼 뽑아내기
tDF.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   survived     891 non-null    int64  
 1   pclass       891 non-null    int64  
 2   sex          891 non-null    object 
 3   age          714 non-null    float64
 4   sibsp        891 non-null    int64  
 5   parch        891 non-null    int64  
 6   fare         891 non-null    float64
 7   embarked     889 non-null    object 
 8   class        891 non-null    object 
 9   who          891 non-null    object 
 10  adult_male   891 non-null    bool   
 11  deck         203 non-null    object 
 12  embark_town  889 non-null    object 
 13  alive        891 non-null    object 
 14  alone        891 non-null    bool   
dtypes: bool(2), float64(2), int64(4), object(7)
memory usage: 92.4+ KB


Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'],
      dtype='object')

# titanic
### **명목형**  
-> 순서 상관 없고, 
- sex	[성별]                    
    * 남/여   	          
- embarked [탑승항구]	           
    * S, C, Q	              
- who [사람유형]	               
    * child / woman / man	  
- adult_male [성인남성여부]	    
    * True / False	  
- deck [갑판:층을의미함]	        
    * 갑판(A/B/C/D/E/F/G/Unknown)	  
- embark_town [탑승도시]	       
    * Southampton / Cherbourg / Queenstown	  
- alone [혼자인지여부]	        
    * True / False	
-> OneHotEncoder 변환 진행 !!

---
### **순서형**  
- pclass [객실등급]	  
    * 1 > 2 > 3	  
- class	[객실등급문자형]	 
    * First > Second > Third	  
-> OrdinalEncoder  변환 진행 !!

---
### **수치형**  
- age [나이]	
    * 결측치 처리 + Scaling(선택)
- sibsp	[동반한 형제/배우자 수]  
    * Scaling 가능
- parch	[동반한 부모/자녀 수]  
    * Scaling 가능
- fare	[요금]	  
    * 변환 X
-> 결측치 처리 + 스케일

---
### **중복 삭제**  
- alive [생존여부]	           
    * Yes/No → 타겟 중복. 삭제  

---
### **Target**  
- survived [생존] 
    * 1/0임으로 변환 X

<hr>



In [ ]:
## OneHotEncoder 함수
def add_onehot_features(df, cols):
    '''
    df       : 원본 DataFrame
    cols     : One-Hot Encoding을 적용할 컬럼 리스트
    return   : One-Hot이 적용된 DateFrame

    '''
    df1 = df.copy()
    ohEncoder = OneHotEncoder(sparse_output=False)
    
    for c in cols:
        # print(c)
        # OneHot 인코딩 학습
        encoded = ohEncoder.fit_transform(df1[[c]])

        # c의 unique갯수만큼 만들어야함
        k = len(df1[c].unique()) # 개수 확인용
        col = [ f'{c}_{i}' for i in df1[c].unique() ]

        # 인코딩 결과를 새로운 DF로 만들기
        encoded_df = pd.DataFrame(encoded,
                                  columns=col,
                                  index=df1.index)
        # print(encoded_df)

        # 원래 컬럼 삭제
        df1.drop(c, axis='columns', inplace=True)
        # display(df1)
        
        # 합치기
        df1 = pd.concat([df1, encoded_df], axis=1)

    return df1

str_of_onehot= ["sex", "alone"]
add_onehot_features(tDF, str_of_onehot)


,survived,pclass,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,sex_male,sex_female,alone_False,alone_True
0,0,3,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,0.0,1.0,1.0,0.0
1,1,1,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,1.0,0.0,1.0,0.0
2,1,3,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,1.0,0.0,0.0,1.0
3,1,1,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,1.0,0.0,1.0,0.0
4,0,3,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,0.0,1.0,0.0,1.0
887,1,1,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,1.0,0.0,0.0,1.0
888,0,3,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,1.0,0.0,1.0,0.0
889,1,1,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,0.0,1.0,0.0,1.0


In [54]:
## OrdinalEncoder 함수
def add_ord_features(df, cols):
    '''
    df       : 원본 DataFrame
    cols     : One-Hot Encoding을 적용할 컬럼 리스트
    return   : One-Hot이 적용된 DateFrame

    '''
    df1 = df.copy()
    ordEncoder = OrdinalEncoder()
    
    for c in cols:
        # 인코딩
        ordEncoder = OrdinalEncoder()
        encoded = ordEncoder.fit_transform(df1[[c]])

        # 그대로 해당 컬럼에 덮어쓰기
        df1[c] = encoded  

    return df1

str_of_ordinal= ["pclass", "class"]
add_ord_features(tDF, str_of_ordinal)


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,2.0,male,22.0,1,0,7.2500,S,2.0,man,True,NaN,Southampton,no,False
1,1,0.0,female,38.0,1,0,71.2833,C,0.0,woman,False,C,Cherbourg,yes,False
2,1,2.0,female,26.0,0,0,7.9250,S,2.0,woman,False,NaN,Southampton,yes,True
3,1,0.0,female,35.0,1,0,53.1000,S,0.0,woman,False,C,Southampton,yes,False
4,0,2.0,male,35.0,0,0,8.0500,S,2.0,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,1.0,male,27.0,0,0,13.0000,S,1.0,man,True,NaN,Southampton,no,True
887,1,0.0,female,19.0,0,0,30.0000,S,0.0,woman,False,B,Southampton,yes,True
888,0,2.0,female,NaN,1,2,23.4500,S,2.0,woman,False,NaN,Southampton,no,False
889,1,0.0,male,26.0,0,0,30.0000,C,0.0,man,True,C,Cherbourg,yes,True


In [ ]:
## 스케일 진행 함수
def add_scaled_features(df, cols):
    """
    df         : 원본
    cols       : 스케일링할 컬럼 이름 리스트
    return     : 새 컬럼이 추가된 DataFrame, Scaled된 DataFrame
    """
    df1 = df.copy() # 원본 두고 새로 만들기
    
    # 스케일러 생성 & 학습
    scaler = StandardScaler()
    scaled_values = scaler.fit_transform(df1[cols])
    
    # 스케일된 컬럼들 DataFrame으로 만들기
    scaled_df = pd.DataFrame(scaled_values, 
                             columns=cols)
    
    # 원본에 cols 제거
    df1.drop(cols, axis='columns', inplace=True)
    
    # cols 제거 df + 스케일된 컬럼들 붙이기
    df_scaled = pd.concat([df1, scaled_df], axis=1)
    
    return df_scaled, scaled_df

## 함수 실행
number_to_scale = ["age", 'sibsp', 'parch', "fare"]
titanic_scaled, scaled_df = add_scaled_features(tDF, number_to_scale)

display(titanic_scaled.head())
display(scaled_df.head())


,survived,pclass,sex,embarked,class,who,adult_male,deck,embark_town,alive,alone,age,sibsp,parch,fare
0,0,3,male,S,Third,man,True,NaN,Southampton,no,False,-0.530377,0.432793,-0.473674,-0.502445
1,1,1,female,C,First,woman,False,C,Cherbourg,yes,False,0.571831,0.432793,-0.473674,0.786845
2,1,3,female,S,Third,woman,False,NaN,Southampton,yes,True,-0.254825,-0.474545,-0.473674,-0.488854
3,1,1,female,S,First,woman,False,C,Southampton,yes,False,0.365167,0.432793,-0.473674,0.420730
4,0,3,male,S,Third,man,True,NaN,Southampton,no,True,0.365167,-0.474545,-0.473674,-0.486337


,age,sibsp,parch,fare
0,-0.530377,0.432793,-0.473674,-0.502445
1,0.571831,0.432793,-0.473674,0.786845
2,-0.254825,-0.474545,-0.473674,-0.488854
3,0.365167,0.432793,-0.473674,0.420730
4,0.365167,-0.474545,-0.473674,-0.486337


In [ ]:
def preprocess_target(y, is_categorical=True):
    """
    y              : Series
    is_categorical : 범주형이면 LabelEncoder, False면 그대로 반환
    """
    if is_categorical:
        le = LabelEncoder()
        y_enc = le.fit_transform(y)
        return y_enc
    else:
        return y.values

target_ret = preprocess_target(tDF['survived'], is_categorical=False)
target_ret



array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,

<hr>

In [58]:
tDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   survived     891 non-null    int64  
 1   pclass       891 non-null    int64  
 2   sex          891 non-null    object 
 3   age          714 non-null    float64
 4   sibsp        891 non-null    int64  
 5   parch        891 non-null    int64  
 6   fare         891 non-null    float64
 7   embarked     889 non-null    object 
 8   class        891 non-null    object 
 9   who          891 non-null    object 
 10  adult_male   891 non-null    bool   
 11  deck         203 non-null    object 
 12  embark_town  889 non-null    object 
 13  alive        891 non-null    object 
 14  alone        891 non-null    bool   
dtypes: bool(2), float64(2), int64(4), object(7)
memory usage: 92.4+ KB


# auto_mpg  

### **명목형**  
-> 순서 상관 없고, ‘이름/범주’ 의미만 있음  

- origin  [제조지역코드]  
    * 1 : USA(미국)  
    * 2 : Europe(유럽)  
    * 3 : Japan(일본)  
    * 숫자지만 “지역 구분”만 의미 → 범주형으로 봄  
    * → OneHotEncoder 변환 진행 !!  

- car name  [차량 이름 / 모델명]  
    * 예: chevrolet chevelle malibu, ford torino, …  
    * 고유 이름(텍스트)로 순서/크기 의미 X  
    * → OneHotEncoder 변환 진행 !!  

---

### **순서형**  
-> “1등급 > 2등급 > 3등급”처럼 순서만 의미가 있는 범주형  

- auto_mpg에서는 **명확한 순서형 범주 컬럼은 없음**  
    * model year : 연도(시간 흐름)라서 보통 **수치형**으로 사용  
    * cylinders : 값이 커질수록 엔진 크기가 커지지만, 일반적으로 **수치형**으로 처리  

-> OrdinalEncoder 변환 진행 X (해당 없음)  

---

### **수치형**  

- cylinders  [실린더 수]  
    * 3, 4, 5, 6, 8 …  
    * 엔진 크기를 대략적으로 표현하는 정수  
    * Scaling 가능  

- displacement  [배기량]  
    * 엔진 체적(배기량), 연속형 숫자  
    * 값 크기 차이가 크므로 Scaling 가능  

- horsepower  [출력(마력)]  
    * 일부 값이 '?' → `np.nan`으로 치환 후  
      float 변환 + 결측치(예: 중앙값) 대체 필요  
    * 그 후 Scaling 가능  

- weight  [차량 중량]  
    * 차량 무게, 값 크기가 큼  
    * 다른 피쳐와 스케일 차이 큼 → Scaling 권장  

- acceleration  [가속 성능(0→60mph 시간)]  
    * 연속형 실수값  
    * Scaling 가능  

- model year  [모델 연식]  
    * 70, 71, 72 … (보통 1900+연도 개념)  
    * 시간 흐름에 따른 변화 표현 → 수치형  
    * Scaling 가능  

-> horsepower 결측/문자 처리 후 + 필요 시 StandardScaler 등으로 스케일 진행 !!  

---

### **중복 삭제 / 특수 처리**  

- 타겟(mpg)와 완전히 같은 의미의 중복 컬럼은 없음  
- 다만 origin은 숫자로 되어 있지만  
    * “지역 코드”일 뿐 → **수치형 X, 범주형 O**  
    * 문자열로 변환 후 OneHotEncoder 적용하는 점이 포인트  

---

### **Target**  

- mpg  [연비 (mile per gallon)]  
    * 연속형 실수값  
    * 회귀(Regression) 문제의 타겟  

-> 수치형 그대로 사용, LabelEncoder 변환 X  
